<a href="https://colab.research.google.com/github/jude1997felix/Decentralized-Air-quality-Monitoring-and-Prediction/blob/main/competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.mlab as mlab
plt.style.use('ggplot')
from matplotlib.pyplot import figure
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MultiLabelBinarizer
import os
import datetime
import math 
import IPython
import IPython.display
import matplotlib as mpl
import tensorflow as tf

In [3]:
def compile_and_fit(model, window, patience=5):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history



In [4]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds


def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels




In [5]:
def time_features(df,time):
  df['weekday'] = df[time].dt.day_name()
  weekdays = ['Monday','Tuesday','Wednesday','Thursday','Friday']
  weekends = ['Saturday','Sunday']
  df['week'] = 'weekday'
  for i in range(len(df.weekday)):
    #print(df.weekday[i])
    #break
    if df.weekday[i] in weekends:
      df['week'][i] = 'weekend'
  # df.head(30)

  y = pd.get_dummies(df.weekday)
  z = pd.get_dummies(df.week)
  del df['weekday']
  del df['week']
  df = df.join(y)
  df = df.merge(z,left_index=True, right_index=True)
  # df = merge(df)

  df['week'] = df[time].dt.isocalendar().week

  df['month'] = df[time].dt.month_name()
  x = pd.get_dummies(df.month)
  df = df.join(x)
  del df['month']
  df['seconds'] = df['Start'].map(datetime.datetime.timestamp)
  day = 24*60*60
  year = (365.2425)*day

  df['Day sin'] = np.sin(df['seconds'] * (2 * np.pi / day))
  df['Day cos'] = np.cos(df['seconds'] * (2 * np.pi / day))
  df['Year sin'] = np.sin(df['seconds'] * (2 * np.pi / year))
  df['Year cos'] = np.cos(df['seconds'] * (2 * np.pi / year))
  # del pm10['Start']
  del df['seconds']
  return df

# pm10 = time_features(pm10,'Start')

 # Data


In [50]:
df = pd.read_csv('/content/imputed_csv_8779.csv',parse_dates=['Start'])
df.head()

,Unnamed: 0,Start,Slut,NO2 (183),NOX as NO2 (184),PM10 (182),PM2.5 (181)
0,0,2015-01-01 00:00:00,2015-01-01 01:00:00,33.3,69.5,101.2,51.9
1,1,2015-01-01 01:00:00,2015-01-01 02:00:00,26.1,39.0,12.5,9.3
2,2,2015-01-01 02:00:00,2015-01-01 03:00:00,18.4,25.7,12.2,9.2
3,3,2015-01-01 03:00:00,2015-01-01 04:00:00,18.1,33.5,16.2,8.9
4,4,2015-01-01 04:00:00,2015-01-01 05:00:00,11.0,22.1,14.5,5.9


In [51]:
del df['Unnamed: 0']
# del df['Black Carbon (410)']
df = time_features(df,'Start')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [52]:
df_copy = df.copy()
df.head()

,Start,Slut,NO2 (183),NOX as NO2 (184),PM10 (182),PM2.5 (181),Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,weekday,weekend,week,April,August,December,February,January,July,June,March,May,November,October,September,Day sin,Day cos,Year sin,Year cos
0,2015-01-01 00:00:00,2015-01-01 01:00:00,33.3,69.5,101.2,51.9,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1.232357e-12,1.000000,0.001505,0.999999
1,2015-01-01 01:00:00,2015-01-01 02:00:00,26.1,39.0,12.5,9.3,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,2.588190e-01,0.965926,0.002222,0.999998
2,2015-01-01 02:00:00,2015-01-01 03:00:00,18.4,25.7,12.2,9.2,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,5.000000e-01,0.866025,0.002939,0.999996
3,2015-01-01 03:00:00,2015-01-01 04:00:00,18.1,33.5,16.2,8.9,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,7.071068e-01,0.707107,0.003656,0.999993
4,2015-01-01 04:00:00,2015-01-01 05:00:00,11.0,22.1,14.5,5.9,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,8.660254e-01,0.500000,0.004372,0.999990


# Model DNN


## Data splitting and windowing


In [53]:
  del df['Start']
  del df['Slut']
  column_indices = {name: i for i, name in enumerate(df.columns)}

  n = len(df)

  cols = df.columns
  x = df.values #returns a numpy array
  sc = StandardScaler()
  x_scaled = sc.fit_transform(x)
  df = pd.DataFrame(x_scaled)

  # y = pm10_test.values
  # y_scaled = sc.fit_transform(y)
  # pm10_test = 
  
  df.columns = cols
  
  # Normalising training data
  train_df = df[0:int(n*0.8)]
  # x = train_df.values
  # train_scaled = sc.fit_transform(x)
  # train_df = pd.DataFrame(train_scaled,columns=cols)
  
  # Normalising validation data
  val_df = df[int(n*0.80):int(n*0.90)]
  # x = val_df.values
  # train_scaled = sc.fit_transform(x)
  # val_df = pd.DataFrame(train_scaled,columns=cols)
  
  # Normalising testing data
  test_df = df[41000:41100]
  # x = test_df.values
  # train_scaled = sc.fit_transform(x)
  # test_df = pd.DataFrame(train_scaled,columns=cols)


  num_features = df.shape[1]
  # return(train_df,val_df,test_df,num_features)
  train_mean = train_df.mean()
  train_std = train_df.std()

  df_std = (df - train_mean) / train_std

In [54]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
                train_df=train_df, val_df=val_df, test_df=test_df,
                label_columns=None):
      # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

      # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                      enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                            enumerate(train_df.columns)}

      # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])


WindowGenerator.split_window = split_window
# WindowGenerator.plot = plot
WindowGenerator.make_dataset = make_dataset

@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
      # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
      # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

  

# single_step_window = WindowGenerator(
#       input_width=2, label_width=1, shift=1,
#       label_columns=['PM10 (107)'])
#   # single_step_window
# print(single_step_window.test)


In [55]:
OUT_STEPS = 24
multi_window = WindowGenerator(input_width=24,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               label_columns=['NO2 (183)',	'NOX as NO2 (184)',	'PM10 (182)',	'PM2.5 (181)'])

# multi_window.plot()
multi_window
#NO2 (183)	NOX as NO2 (184)	PM10 (182)	PM2.5 (181)

Total window size: 48
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
Label column name(s): ['NO2 (183)', 'NOX as NO2 (184)', 'PM10 (182)', 'PM2.5 (181)']

## Model training and prediction using DNN

In [56]:
MAX_EPOCHS = 20
multi_dense_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, dense_units]
    tf.keras.layers.Dense(512, activation='relu'),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*4,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, 4])
])

history = compile_and_fit(multi_dense_model, multi_window)

# IPython.display.clear_output()
# multi_val_performance['Dense'] = multi_dense_model.evaluate(multi_window.val)
# multi_performance = multi_dense_model.predict(multi_window.test)
# multi_window.plot(multi_dense_model)

Epoch 1/20
1095/1095 [==============================] - 5s 5ms/step - loss: 0.6522 - mean_absolute_error: 0.5480 - val_loss: 0.8857 - val_mean_absolute_error: 0.6190
Epoch 2/20
1095/1095 [==============================] - 5s 5ms/step - loss: 0.6066 - mean_absolute_error: 0.5263 - val_loss: 0.8688 - val_mean_absolute_error: 0.6150
Epoch 3/20
1095/1095 [==============================] - 5s 5ms/step - loss: 0.5899 - mean_absolute_error: 0.5182 - val_loss: 0.8660 - val_mean_absolute_error: 0.6057
Epoch 4/20
1095/1095 [==============================] - 5s 5ms/step - loss: 0.5762 - mean_absolute_error: 0.5117 - val_loss: 0.8602 - val_mean_absolute_error: 0.5965
Epoch 5/20
1095/1095 [==============================] - 5s 5ms/step - loss: 0.5667 - mean_absolute_error: 0.5071 - val_loss: 0.8537 - val_mean_absolute_error: 0.6033
Epoch 6/20
1095/1095 [==============================] - 5s 5ms/step - loss: 0.5598 - mean_absolute_error: 0.5042 - val_loss: 0.8617 - val_mean_absolute_error: 0.6058
Epoc

In [57]:
predictions = dict()
for i in range(92):
  idx = 41591  + i*24
  test_df = df.iloc[idx : idx+48]
  
  class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
                  train_df=train_df, val_df=val_df, test_df=test_df,
                  label_columns=None):
        # Store the raw data.
      self.train_df = train_df
      self.val_df = val_df
      self.test_df = test_df

        # Work out the label column indices.
      self.label_columns = label_columns
      if label_columns is not None:
        self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
      self.column_indices = {name: i for i, name in
                              enumerate(train_df.columns)}

        # Work out the window parameters.
      self.input_width = input_width
      self.label_width = label_width
      self.shift = shift

      self.total_window_size = input_width + shift

      self.input_slice = slice(0, input_width)
      self.input_indices = np.arange(self.total_window_size)[self.input_slice]

      self.label_start = self.total_window_size - self.label_width
      self.labels_slice = slice(self.label_start, None)
      self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
      return '\n'.join([
          f'Total window size: {self.total_window_size}',
          f'Input indices: {self.input_indices}',
          f'Label indices: {self.label_indices}',
          f'Label column name(s): {self.label_columns}'])


  WindowGenerator.split_window = split_window
  # WindowGenerator.plot = plot
  WindowGenerator.make_dataset = make_dataset

  @property
  def train(self):
    return self.make_dataset(self.train_df)

  @property
  def val(self):
    return self.make_dataset(self.val_df)

  @property
  def test(self):
    return self.make_dataset(self.test_df)

  @property
  def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, '_example', None)
    if result is None:
        # No example batch was found, so get one from the `.train` dataset
      result = next(iter(self.train))
        # And cache it for next time
      self._example = result
    return result

  WindowGenerator.train = train
  WindowGenerator.val = val
  WindowGenerator.test = test
  WindowGenerator.example = example


  OUT_STEPS = 24
  multi_window = WindowGenerator(input_width=24,
                                label_width=OUT_STEPS,
                                shift=OUT_STEPS,
                                label_columns=['NO2 (183)',	'NOX as NO2 (184)',	'PM10 (182)',	'PM2.5 (181)'])

  # multi_window.plot()
  # multi_window


  pred = multi_dense_model.predict(multi_window.test)
  pred = pred.reshape(24,4)
  predictions[i] = pred
idx

43775

In [58]:
len(predictions)

92

## Denormalizing and calculating the smape values

In [59]:
idx = 41616
for days in range(92):
  for hour in range(24):
    for feat in range(4):
      if feat == 0:
        df['NO2 (183)'].iloc[idx] = predictions[days][hour][feat]
      elif feat == 1:
        df['NOX as NO2 (184)'].iloc[idx] = predictions[days][hour][feat]
      elif feat == 2:
        df['PM10 (182)'].iloc[idx] = predictions[days][hour][feat]
      else:
        df['PM2.5 (181)'].iloc[idx] = predictions[days][hour][feat]
    idx = idx+1
# label_columns=['NO2 (115)',	'NOX as NO2 (114)',	'PM10 (116)',	'PM2.5 (111)']

In [60]:
x = df.values
x_scaled = sc.inverse_transform(x)
df_norm = pd.DataFrame(x_scaled, columns=cols)
df_norm.head()

,NO2 (183),NOX as NO2 (184),PM10 (182),PM2.5 (181),Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,weekday,weekend,week,April,August,December,February,January,July,June,March,May,November,October,September,Day sin,Day cos,Year sin,Year cos
0,33.3,69.5,101.2,51.9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.232357e-12,1.000000,0.001505,0.999999
1,26.1,39.0,12.5,9.3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.588190e-01,0.965926,0.002222,0.999998
2,18.4,25.7,12.2,9.2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.000000e-01,0.866025,0.002939,0.999996
3,18.1,33.5,16.2,8.9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.071068e-01,0.707107,0.003656,0.999993
4,11.0,22.1,14.5,5.9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.660254e-01,0.500000,0.004372,0.999990


In [61]:
del df_copy['Slut']
del df_copy['Start']
df_copy.head()

,NO2 (183),NOX as NO2 (184),PM10 (182),PM2.5 (181),Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,weekday,weekend,week,April,August,December,February,January,July,June,March,May,November,October,September,Day sin,Day cos,Year sin,Year cos
0,33.3,69.5,101.2,51.9,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1.232357e-12,1.000000,0.001505,0.999999
1,26.1,39.0,12.5,9.3,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,2.588190e-01,0.965926,0.002222,0.999998
2,18.4,25.7,12.2,9.2,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,5.000000e-01,0.866025,0.002939,0.999996
3,18.1,33.5,16.2,8.9,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,7.071068e-01,0.707107,0.003656,0.999993
4,11.0,22.1,14.5,5.9,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,8.660254e-01,0.500000,0.004372,0.999990


In [62]:
y_test = df_copy[41616:]
y_pred = df_norm[41616:]

In [65]:
y_test.head()

,NO2 (183),NOX as NO2 (184),PM10 (182),PM2.5 (181),Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,weekday,weekend,week,April,August,December,February,January,July,June,March,May,November,October,September,Day sin,Day cos,Year sin,Year cos
41616,8.1,13.8,0.9,0.0,0,0,0,0,0,1,0,1,0,40,0,0,0,0,0,0,0,0,0,0,1,0,-1.023418e-11,1.000000,-0.999902,-0.014009
41617,7.4,12.6,2.6,0.7,0,0,0,0,0,1,0,1,0,40,0,0,0,0,0,0,0,0,0,0,1,0,2.588190e-01,0.965926,-0.999912,-0.013292
41618,8.7,14.7,1.5,0.0,0,0,0,0,0,1,0,1,0,40,0,0,0,0,0,0,0,0,0,0,1,0,5.000000e-01,0.866025,-0.999921,-0.012576
41619,7.9,15.0,2.3,0.0,0,0,0,0,0,1,0,1,0,40,0,0,0,0,0,0,0,0,0,0,1,0,7.071068e-01,0.707107,-0.999930,-0.011859
41620,5.5,14.4,1.2,0.0,0,0,0,0,0,1,0,1,0,40,0,0,0,0,0,0,0,0,0,0,1,0,8.660254e-01,0.500000,-0.999938,-0.011142


In [66]:
y_pred.head()

,NO2 (183),NOX as NO2 (184),PM10 (182),PM2.5 (181),Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,weekday,weekend,week,April,August,December,February,January,July,June,March,May,November,October,September,Day sin,Day cos,Year sin,Year cos
41616,9.565182,22.012958,2.949195,0.179437,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.023418e-11,1.000000,-0.999902,-0.014009
41617,6.401556,15.304794,1.929859,0.762254,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.588190e-01,0.965926,-0.999912,-0.013292
41618,4.940440,4.320073,1.808226,1.053698,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.000000e-01,0.866025,-0.999921,-0.012576
41619,3.220639,2.769098,-0.478331,0.916951,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.071068e-01,0.707107,-0.999930,-0.011859
41620,0.088982,-5.294693,-0.778785,1.372186,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.660254e-01,0.500000,-0.999938,-0.011142


In [85]:
col = 'NOX as NO2 (184)'

In [86]:
negative_index = []
for idx,val in enumerate(y_pred[col]):
  if val < 0:
    negative_index.append(idx)

print(len(negative_index))

79


In [87]:
y_pred[col] = y_pred[col].apply(lambda x: abs(x*0) if x<0 else x)

negative_index = []
for idx,val in enumerate(y_pred[col]):
  if val < 0:
    negative_index.append(idx)

print(len(negative_index))

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [88]:
def smape(y_true, y_pred):
  return tf.reduce_mean(2 * tf.abs(y_true - y_pred)
                          / (tf.abs(y_pred) + tf.abs(y_true)) , axis=-1)


In [89]:
smape(y_test['NO2 (183)'],y_pred['NO2 (183)'])

<tf.Tensor: shape=(), dtype=float64, numpy=0.51868599561753>

In [90]:
smape(y_test['NOX as NO2 (184)'],y_pred['NOX as NO2 (184)'])

<tf.Tensor: shape=(), dtype=float64, numpy=0.7180070422380903>

In [70]:
smape(y_test['PM10 (182)'],y_pred['PM10 (182)'])

<tf.Tensor: shape=(), dtype=float64, numpy=0.49148945626608126>

In [71]:
smape(y_test['PM2.5 (181)'],y_pred['PM2.5 (181)'])

<tf.Tensor: shape=(), dtype=float64, numpy=0.5692471260605517>

In [666]:
(0.48324000238497156 + 0.6622102969509207 + 0.4914233785283172 + 0.565974910449472)/4

0.5507121470784204

In [667]:
# y_pred.to_csv("test_values_8779.csv")

In [664]:
len(y_pred)

2208

In [665]:
y_pred.head()

,NO2 (183),NOX as NO2 (184),PM10 (182),PM2.5 (181),Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,weekday,weekend,week,April,August,December,February,January,July,June,March,May,November,October,September,Day sin,Day cos,Year sin,Year cos
41616,11.094571,15.664850,4.798539,0.125053,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.023418e-11,1.000000,-0.999902,-0.014009
41617,9.478485,15.904051,6.392095,0.363726,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.588190e-01,0.965926,-0.999912,-0.013292
41618,8.585893,10.276290,7.858934,1.088547,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.000000e-01,0.866025,-0.999921,-0.012576
41619,7.697448,7.251410,4.388444,1.322334,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.071068e-01,0.707107,-0.999930,-0.011859
41620,5.628818,0.662203,2.755216,1.156763,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.660254e-01,0.500000,-0.999938,-0.011142


# Model LSTM


## Data splitting and windowing


In [ ]:
  del df['Start']
  del df['Slut']
  column_indices = {name: i for i, name in enumerate(df.columns)}

  n = len(df)

  cols = df.columns
  x = df.values #returns a numpy array
  sc = StandardScaler()
  x_scaled = sc.fit_transform(x)
  df = pd.DataFrame(x_scaled)

  # y = pm10_test.values
  # y_scaled = sc.fit_transform(y)
  # pm10_test = 
  
  df.columns = cols
  
  # Normalising training data
  train_df = df[0:int(n*0.8)]
  # x = train_df.values
  # train_scaled = sc.fit_transform(x)
  # train_df = pd.DataFrame(train_scaled,columns=cols)
  
  # Normalising validation data
  val_df = df[int(n*0.80):int(n*0.90)]
  # x = val_df.values
  # train_scaled = sc.fit_transform(x)
  # val_df = pd.DataFrame(train_scaled,columns=cols)
  
  # Normalising testing data
  test_df = df[41000:41100]
  # x = test_df.values
  # train_scaled = sc.fit_transform(x)
  # test_df = pd.DataFrame(train_scaled,columns=cols)


  num_features = df.shape[1]
  # return(train_df,val_df,test_df,num_features)
  train_mean = train_df.mean()
  train_std = train_df.std()

  df_std = (df - train_mean) / train_std

In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
                train_df=train_df, val_df=val_df, test_df=test_df,
                label_columns=None):
      # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

      # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                      enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                            enumerate(train_df.columns)}

      # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])


WindowGenerator.split_window = split_window
# WindowGenerator.plot = plot
WindowGenerator.make_dataset = make_dataset

@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
      # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
      # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

  

# single_step_window = WindowGenerator(
#       input_width=2, label_width=1, shift=1,
#       label_columns=['PM10 (107)'])
#   # single_step_window
# print(single_step_window.test)


In [ ]:
OUT_STEPS = 24
multi_window = WindowGenerator(input_width=24,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS)

# multi_window.plot()
multi_window

Total window size: 48
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
Label column name(s): None

## Model training and prediction using LSTM


In [ ]:
MAX_EPOCHS = 20
multi_dense_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, dense_units]
    tf.keras.layers.Dense(512, activation='relu'),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_dense_model, multi_window)

# IPython.display.clear_output()
# multi_val_performance['Dense'] = multi_dense_model.evaluate(multi_window.val)
# multi_performance = multi_dense_model.predict(multi_window.test)
# multi_window.plot(multi_dense_model)

Epoch 1/20
1095/1095 [==============================] - 6s 5ms/step - loss: 0.8283 - mean_absolute_error: 0.6336 - val_loss: 0.8594 - val_mean_absolute_error: 0.6585
Epoch 2/20
1095/1095 [==============================] - 5s 5ms/step - loss: 0.8136 - mean_absolute_error: 0.6264 - val_loss: 0.8560 - val_mean_absolute_error: 0.6551
Epoch 3/20
1095/1095 [==============================] - 5s 5ms/step - loss: 0.8103 - mean_absolute_error: 0.6248 - val_loss: 0.8578 - val_mean_absolute_error: 0.6576
Epoch 4/20
1095/1095 [==============================] - 5s 5ms/step - loss: 0.8079 - mean_absolute_error: 0.6237 - val_loss: 0.8548 - val_mean_absolute_error: 0.6556
Epoch 5/20
1095/1095 [==============================] - 5s 5ms/step - loss: 0.8062 - mean_absolute_error: 0.6232 - val_loss: 0.8580 - val_mean_absolute_error: 0.6598
Epoch 6/20
1095/1095 [==============================] - 5s 5ms/step - loss: 0.8050 - mean_absolute_error: 0.6228 - val_loss: 0.8548 - val_mean_absolute_error: 0.6556
Epoc

In [ ]:
predictions = dict()
for i in range(92):
  idx = 41591  + i*24
  test_df = df.iloc[idx : idx+48]
  
  class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
                  train_df=train_df, val_df=val_df, test_df=test_df,
                  label_columns=None):
        # Store the raw data.
      self.train_df = train_df
      self.val_df = val_df
      self.test_df = test_df

        # Work out the label column indices.
      self.label_columns = label_columns
      if label_columns is not None:
        self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
      self.column_indices = {name: i for i, name in
                              enumerate(train_df.columns)}

        # Work out the window parameters.
      self.input_width = input_width
      self.label_width = label_width
      self.shift = shift

      self.total_window_size = input_width + shift

      self.input_slice = slice(0, input_width)
      self.input_indices = np.arange(self.total_window_size)[self.input_slice]

      self.label_start = self.total_window_size - self.label_width
      self.labels_slice = slice(self.label_start, None)
      self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
      return '\n'.join([
          f'Total window size: {self.total_window_size}',
          f'Input indices: {self.input_indices}',
          f'Label indices: {self.label_indices}',
          f'Label column name(s): {self.label_columns}'])


  WindowGenerator.split_window = split_window
  # WindowGenerator.plot = plot
  WindowGenerator.make_dataset = make_dataset

  @property
  def train(self):
    return self.make_dataset(self.train_df)

  @property
  def val(self):
    return self.make_dataset(self.val_df)

  @property
  def test(self):
    return self.make_dataset(self.test_df)

  @property
  def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, '_example', None)
    if result is None:
        # No example batch was found, so get one from the `.train` dataset
      result = next(iter(self.train))
        # And cache it for next time
      self._example = result
    return result

  WindowGenerator.train = train
  WindowGenerator.val = val
  WindowGenerator.test = test
  WindowGenerator.example = example


  OUT_STEPS = 24
  multi_window = WindowGenerator(input_width=24,
                                label_width=OUT_STEPS,
                                shift=OUT_STEPS)

  # multi_window.plot()
  # multi_window


  pred = multi_dense_model.predict(multi_window.test)
  pred = pred.reshape(24,4)
  predictions[i] = pred
idx

43775

In [ ]:
test_df.shape

(48, 4)

## Denormalizing and calculating the smape values

In [ ]:
idx = 41616
for days in range(92):
  for hour in range(24):
    for feat in range(4):
      if feat == 0:
        df['NO2 (344)'].iloc[idx] = predictions[days][hour][feat]
      elif feat == 1:
        df['NOX as NO2 (345)'].iloc[idx] = predictions[days][hour][feat]
      elif feat == 2:
        df['PM10 (342)'].iloc[idx] = predictions[days][hour][feat]
      else:
        df['PM2.5 (343)'].iloc[idx] = predictions[days][hour][feat]
    idx = idx+1


In [ ]:
x = df.values
x_scaled = sc.inverse_transform(x)
df_norm = pd.DataFrame(x_scaled, columns=cols)
df_norm.head()

,NO2 (344),NOX as NO2 (345),PM10 (342),PM2.5 (343)
0,11.0,18.8,27.9,19.8
1,15.3,25.0,15.5,4.5
2,13.4,19.8,17.1,4.6
3,9.6,13.8,15.2,5.3
4,3.5,5.8,16.3,6.6


In [ ]:
del df_copy['Slut']
del df_copy['Start']
df_copy.head()

,NO2 (344),NOX as NO2 (345),PM10 (342),PM2.5 (343)
0,11.0,18.8,27.9,19.8
1,15.3,25.0,15.5,4.5
2,13.4,19.8,17.1,4.6
3,9.6,13.8,15.2,5.3
4,3.5,5.8,16.3,6.6


In [ ]:
y_test = df_copy[41616:]
y_pred = df_norm[41616:]

In [ ]:
y_test.head()

,NO2 (344),NOX as NO2 (345),PM10 (342),PM2.5 (343)
41616,4.7,7.7,2.0,0.000000
41617,1.8,3.2,2.3,0.000000
41618,1.3,1.9,2.2,0.053142
41619,3.5,6.0,3.8,0.400000
41620,4.8,8.7,1.4,0.000000


In [ ]:
y_pred.head()

,NO2 (344),NOX as NO2 (345),PM10 (342),PM2.5 (343)
41616,20.514492,60.672230,4.616429,0.613217
41617,24.376423,73.177984,5.618655,0.974472
41618,26.932653,80.688688,6.089956,1.447461
41619,29.497219,87.931756,7.959068,1.634097
41620,30.592504,89.146453,9.615379,2.016642


In [ ]:
def smape(y_true, y_pred):
  return tf.reduce_mean(2 * tf.abs(y_true - y_pred)
                          / (tf.abs(y_pred) + tf.abs(y_true)) , axis=-1)


In [ ]:
smape(y_test['NO2 (344)'],y_pred['NO2 (344)'])

<tf.Tensor: shape=(), dtype=float64, numpy=0.4919369297002677>

In [ ]:
smape(y_test['NOX as NO2 (345)'],y_pred['NOX as NO2 (345)'])

<tf.Tensor: shape=(), dtype=float64, numpy=0.599664731010258>

In [ ]:
smape(y_test['PM10 (342)'],y_pred['PM10 (342)'])

<tf.Tensor: shape=(), dtype=float64, numpy=0.5512700275253708>

In [ ]:
smape(y_test['PM2.5 (343)'],y_pred['PM2.5 (343)'])

<tf.Tensor: shape=(), dtype=float64, numpy=0.5721971259400087>

In [ ]:
(0.4801 + 0.5882 + 0.5592 + 0.5594)/4

0.546725

In [ ]:
y_pred.to_csv("test_values_18644.csv")

In [ ]:
len(y_pred)

2209

In [ ]:
y_pred.head()

,NO2 (344),NOX as NO2 (345),PM10 (342),PM2.5 (343)
41615,19.471270,60.659505,5.046160,0.296953
41616,24.767853,79.357072,7.662711,1.024119
41617,27.597073,88.657137,9.913626,1.251994
41618,29.104562,91.542055,10.836983,1.530252
41619,31.074538,96.405039,12.291671,1.982532


# For fun

In [47]:
df = pd.read_csv("/content/test_values_18644.csv")

In [38]:
del df['Unnamed: 0']
df.head()

,Start,NO2,NOX as NO2,PM10,PM2.5
0,2019-10-01 00:00:00,13.411764,37.775188,3.255836,0.249491
1,2019-10-01 01:00:00,9.858790,27.435930,8.285621,0.564187
2,2019-10-01 02:00:00,5.639668,18.976900,11.349111,0.268150
3,2019-10-01 03:00:00,5.538182,18.482940,12.547909,0.746775
4,2019-10-01 04:00:00,7.793204,25.944860,8.504370,1.058651


In [29]:
# df = df[:-1]

In [30]:
df['Start'] = pd.date_range(start='2019-10-01 00:00:00', periods=len(df), freq='H')
df.head()

,NO2 (344),NOX as NO2 (345),PM10 (342),PM2.5 (343),Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,weekday,weekend,week,April,August,December,February,January,July,June,March,May,November,October,September,Day sin,Day cos,Year sin,Year cos,Start
0,13.411764,37.775188,3.255836,0.249491,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.023418e-11,1.000000,-0.999902,-0.014009,2019-10-01 00:00:00
1,9.858790,27.435930,8.285621,0.564187,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.588190e-01,0.965926,-0.999912,-0.013292,2019-10-01 01:00:00
2,5.639668,18.976900,11.349111,0.268150,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.000000e-01,0.866025,-0.999921,-0.012576,2019-10-01 02:00:00
3,5.538182,18.482940,12.547909,0.746775,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.071068e-01,0.707107,-0.999930,-0.011859,2019-10-01 03:00:00
4,7.793204,25.944860,8.504370,1.058651,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.660254e-01,0.500000,-0.999938,-0.011142,2019-10-01 04:00:00


In [31]:
df.rename(columns={'NO2 (344)': 'NO2', 'NOX as NO2 (345)': 'NOX as NO2','PM10 (342)':'PM10','PM2.5 (343)':'PM2.5'}, inplace=True)
df.head()

,NO2,NOX as NO2,PM10,PM2.5,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,weekday,weekend,week,April,August,December,February,January,July,June,March,May,November,October,September,Day sin,Day cos,Year sin,Year cos,Start
0,13.411764,37.775188,3.255836,0.249491,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.023418e-11,1.000000,-0.999902,-0.014009,2019-10-01 00:00:00
1,9.858790,27.435930,8.285621,0.564187,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.588190e-01,0.965926,-0.999912,-0.013292,2019-10-01 01:00:00
2,5.639668,18.976900,11.349111,0.268150,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.000000e-01,0.866025,-0.999921,-0.012576,2019-10-01 02:00:00
3,5.538182,18.482940,12.547909,0.746775,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.071068e-01,0.707107,-0.999930,-0.011859,2019-10-01 03:00:00
4,7.793204,25.944860,8.504370,1.058651,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.660254e-01,0.500000,-0.999938,-0.011142,2019-10-01 04:00:00


In [32]:
df = df[['Start','NO2','NOX as NO2','PM10','PM2.5']]
df.head()

,Start,NO2,NOX as NO2,PM10,PM2.5
0,2019-10-01 00:00:00,13.411764,37.775188,3.255836,0.249491
1,2019-10-01 01:00:00,9.858790,27.435930,8.285621,0.564187
2,2019-10-01 02:00:00,5.639668,18.976900,11.349111,0.268150
3,2019-10-01 03:00:00,5.538182,18.482940,12.547909,0.746775
4,2019-10-01 04:00:00,7.793204,25.944860,8.504370,1.058651


In [33]:
df.to_csv('test_values_18644.csv')

In [39]:
df_true = pd.read_csv('/content/imputed_csv_18644.csv')

In [40]:
df_true.head()

,Unnamed: 0,Start,Slut,NO2 (344),NOX as NO2 (345),PM10 (342),PM2.5 (343)
0,0,2015-01-01 00:00:00,2015-01-01 01:00:00,11.0,18.8,27.9,19.8
1,1,2015-01-01 01:00:00,2015-01-01 02:00:00,15.3,25.0,15.5,4.5
2,2,2015-01-01 02:00:00,2015-01-01 03:00:00,13.4,19.8,17.1,4.6
3,3,2015-01-01 03:00:00,2015-01-01 04:00:00,9.6,13.8,15.2,5.3
4,4,2015-01-01 04:00:00,2015-01-01 05:00:00,3.5,5.8,16.3,6.6


In [41]:
del df_true['Unnamed: 0']
# del df_true['Start']
del df_true['Slut']
df_true.head()

,Start,NO2 (344),NOX as NO2 (345),PM10 (342),PM2.5 (343)
0,2015-01-01 00:00:00,11.0,18.8,27.9,19.8
1,2015-01-01 01:00:00,15.3,25.0,15.5,4.5
2,2015-01-01 02:00:00,13.4,19.8,17.1,4.6
3,2015-01-01 03:00:00,9.6,13.8,15.2,5.3
4,2015-01-01 04:00:00,3.5,5.8,16.3,6.6


In [42]:
df_true = df_true[41616:]
len(df_true)

2208

In [43]:
df_true.index = range(len(df_true))

In [46]:
len(df)

2207

In [44]:
def smape(y_true, y_pred):
  return tf.reduce_mean(2 * tf.abs(y_true - y_pred)
                          / (tf.abs(y_pred) + tf.abs(y_true)) , axis=-1)


In [45]:
smape(df_true['NO2 (344)'],df['NO2'])

InvalidArgumentError: ignored

In [ ]:
smape(df_true['NOX as NO2 (103)'],df['NOX as NO2'])

<tf.Tensor: shape=(), dtype=float64, numpy=0.6230520869997159>

In [ ]:
smape(df_true['PM10 (107)'],df['PM10'])

<tf.Tensor: shape=(), dtype=float64, numpy=0.5224191968378576>

In [ ]:
smape(df_true['PM2.5 (102)'],df['PM2.5'])

<tf.Tensor: shape=(), dtype=float64, numpy=0.4841744412177632>

In [ ]:
(0.5743716488572558 + 0.6230520869997159 + 0.5224191968378576+ 0.4841744412177632)/4

0.5510043434781481

In [34]:
df.head()

,Start,NO2,NOX as NO2,PM10,PM2.5
0,2019-10-01 00:00:00,13.411764,37.775188,3.255836,0.249491
1,2019-10-01 01:00:00,9.858790,27.435930,8.285621,0.564187
2,2019-10-01 02:00:00,5.639668,18.976900,11.349111,0.268150
3,2019-10-01 03:00:00,5.538182,18.482940,12.547909,0.746775
4,2019-10-01 04:00:00,7.793204,25.944860,8.504370,1.058651


In [49]:
df

,Unnamed: 0,Start,NO2,NOX as NO2,PM10,PM2.5
0,0,2019-10-01 00:00:00,13.411764,37.775188,3.255836,0.249491
1,1,2019-10-01 01:00:00,9.858790,27.435930,8.285621,0.564187
2,2,2019-10-01 02:00:00,5.639668,18.976900,11.349111,0.268150
3,3,2019-10-01 03:00:00,5.538182,18.482940,12.547909,0.746775
4,4,2019-10-01 04:00:00,7.793204,25.944860,8.504370,1.058651
...,...,...,...,...,...,...
2202,2202,2019-12-31 18:00:00,40.194453,111.171733,4.860192,2.103264
2203,2203,2019-12-31 19:00:00,37.853666,100.128184,5.348709,2.648363
2204,2204,2019-12-31 20:00:00,35.199660,86.070787,2.685396,2.371309
2205,2205,2019-12-31 21:00:00,34.870944,85.362804,-0.181894,2.033602
